In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [2]:
# Loading train and test data

train = pd.read_csv("../Dataset/train.csv") 
test = pd.read_csv("../Dataset/test.csv") 


In [3]:
# Balancing Data

train = train[train["TARGET"] == 0].sample(3008).append(train[train["TARGET"] == 1])

In [4]:
# Removing variables that represents the same

train_corr = train.corr(method = 'pearson')
train_corr = np.array(train_corr.fillna(0))
aux = []

for i in range(len(train.columns)-1):
    for j in range(i+1, len(train_corr)-1):
            if abs(train_corr[i][j]) > 0.90 and j not in aux:
                aux.append(j)
                
cols = train.columns[aux]
train = train.drop(cols, axis = 1)
test = test.drop(cols, axis = 1)

In [5]:
# Getting columns with low correlation and applying PCA

train_corr = train.corr(method = 'spearman')["TARGET"]
train_corr = train_corr.fillna(0)
trainPCA = train
testPCA = test

i = len(train_corr) - 2

while i >= 0:
    if (abs(train_corr[i]) < 0.10) or (abs(train_corr[i]) > 0.20):
        trainPCA = trainPCA.drop(train_corr.index[i],1)
        testPCA = testPCA.drop(train_corr.index[i],1)
    i -= 1
trainPCA = trainPCA.drop("TARGET",1)

# Applying PCA

pca = PCA(n_components=3)
trainPCA = pd.DataFrame(pca.fit_transform(trainPCA))
testPCA = pd.DataFrame(pca.fit_transform(testPCA))

In [6]:
# Removing columns with correlation lower than 0.13 to the target variable

train_corr = train.corr(method = 'pearson')["TARGET"]
train_corr = train_corr.fillna(0)

i = len(train_corr) - 2

while i >= 0:
    if abs(train_corr[i]) < 0.20:
        col = train_corr.index[i]
        train = train.drop(col,1)
        test = test.drop(col,1)
    i -= 1

In [ ]:
# Making var15 levels

def get_level(x):
    if x > 19 and x < 26:
        y = 1
    else: y = 0
    return y

train["var15_2"] = train["var15"].apply(get_level)
test["var15_2"] = test["var15"].apply(get_level)

In [ ]:
# Merging data

train["PCA1"] = trainPCA[0].values
train["PCA2"] = trainPCA[1].values
train["PCA3"] = trainPCA[2].values
#train["PCA4"] = trainPCA[3].values

test["PCA1"] = testPCA[0].values
test["PCA2"] = testPCA[1].values
test["PCA3"] = testPCA[2].values
#test["PCA4"] = testPCA[3].values

In [ ]:
# Removing outliers

train = train[(np.abs(stats.zscore(train)) < 3).all(axis=1)]

In [ ]:
# Splitting data

Y = train["TARGET"]    
X = train.drop("TARGET",1)

In [ ]:
# Normalizing Data

x = X.values
min_max_scaler = preprocessing.MinMaxScaler(feature_range = (0, 1))
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled, columns = X.columns)

x = test.values
min_max_scaler = preprocessing.MinMaxScaler(feature_range = (0, 1))
x_scaled = min_max_scaler.fit_transform(x)
test = pd.DataFrame(x_scaled, columns = test.columns)

In [ ]:
X.describe()

In [ ]:
#Training Model - Random Forest - 0.65 Accuracy

num_trees = 200
max_features = 5
model = RandomForestClassifier(n_estimators = num_trees, max_features = max_features)
model.fit(X, Y.values.ravel())

model.feature_importances_

In [ ]:
#Training Model - Logistic Regression - 0.70 Accuracy

model = LogisticRegression()
result = model.fit(X, Y)

# Runing the model on test data

result = pd.DataFrame(model.predict(test), columns = ["TARGET"])

result.index.name = "ID"
ID = pd.read_csv("../Dataset/test.csv")
result.index = ID["ID"]
result.to_csv("../Dataset/Kaggle.csv")